# ETL Project Report

Extraction: We obtained both data samples from Kaggle.com. 

    1st data set format: CSV
    2nd data set format: CSV

Cleaning and Trasnformation: 

1st Data Set: 
    1. Took out extra columns that we didn't need
    2. Parse just the borough name out of the first column
    3. Group by the Borough column to get total count of participants and public assistance users
    4. Calculated the percentage of people on public assitance and formatted the percentages
    5. Created a new DataFrame with a reset index
    
2nd Data Set
    1. Took out uneeded columns
    
Loading Data:
    We loaded the data into 2 different MySQL tables, becuase we knew what kind of data we were dealing with
    and it was already well structured/cleaned. We then merged the two tables on their common column which
    was 'Borough'.
    
    

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [24]:
#Transform into a DataFrame
hood_breakdown = "neighborhood-development-area-breakdowns.csv"
hood_breakdown = pd.read_csv(hood_breakdown)

#change column name
boro_breakdown = hood_breakdown[["JURISDICTION NAME", "COUNT PARTICIPANTS", "COUNT RECEIVES PUBLIC ASSISTANCE"]].copy()
boro_breakdown = boro_breakdown.rename(columns={"JURISDICTION NAME": "BOROUGH"})



#add column for participant total
# boro_breakdown["PARTICIPANT TOTAL"] = boro_breakdown["COUNT PARTICIPANT"].sum()

boro_breakdown.head(20)




,BOROUGH,COUNT PARTICIPANTS,COUNT RECEIVES PUBLIC ASSISTANCE
0,Bronx Neighborhood Development Area 001,59,20
1,Bronx Neighborhood Development Area 002,7,3
2,Bronx Neighborhood Development Area 003,15,6
3,Bronx Neighborhood Development Area 004,44,13
4,Bronx Neighborhood Development Area 005,25,12
5,Bronx Neighborhood Development Area 006,15,5
6,Bronx Neighborhood Development Area 007,207,67
7,Bronx Neighborhood Development Area 008,61,21
8,Bronx Neighborhood Development Area 009,0,0
9,Bronx Neighborhood Development Area 010,33,13


In [16]:
# new data frame with split value columns
new = boro_breakdown["BOROUGH"].str.split(" ", n = 1, expand = True)

# making separate first name column from new data frame
boro_breakdown["BOROUGH"]= new[0]

#Group by borough
hood_group = boro_breakdown.groupby(["BOROUGH"])
group_hood = hood_group.count()
group_hood["TOTAL PARTICIPANTS"] = hood_group["COUNT PARTICIPANTS"].sum()

#add columns for total amount on public assitance, and their percentage of the population
group_hood["TOTAL PUBLIC ASSISTANCE"] = hood_group["COUNT RECEIVES PUBLIC ASSISTANCE"].sum()
group_hood["% RECEIVES PUBLIC ASSITANCE"] = group_hood["TOTAL PUBLIC ASSISTANCE"]/group_hood["TOTAL PARTICIPANTS"]

#rename columns
group_hood = group_hood.rename(columns={"TOTAL PARTICIPANTS": "Total_Participants", "TOTAL PUBLIC ASSISTANCE":
                                       "Total_Pub_Assist", "% RECEIVES PUBLIC ASSITANCE": "Percent_Pub_Assist"})
group_hood = group_hood.drop(columns=["COUNT PARTICIPANTS", "COUNT RECEIVES PUBLIC ASSISTANCE"])

#format percentages
group_hood["Percent_Pub_Assist"] = group_hood["Percent_Pub_Assist"]*100
group_hood["Percent_Pub_Assist"] = group_hood["Percent_Pub_Assist"].map("{:.1f}".format)

#reset index, create new df so its not a groupby
new_group = group_hood.reset_index()
new_group

,BOROUGH,Total_Participants,Total_Pub_Assist,Percent_Pub_Assist
0,Bronx,556,187,33.6
1,Brooklyn,1067,360,33.7
2,Manhattan,127,30,23.6
3,Not,2338,824,35.2
4,Queens,48,5,10.4


In [20]:
#Transform to DataFrame
crime_breakdown = "NYPD_Complaint_Data_Historic.csv"
crime_breakdown = pd.read_csv(crime_breakdown)

#take out only the columns we need and make a new df
new_crime_pd = crime_breakdown[["BORO_NM", "LAW_CAT_CD", "OFNS_DESC"]].copy()

new_crime_pd.head(20)

,BORO_NM,LAW_CAT_CD,OFNS_DESC
0,BRONX,FELONY,FORGERY
1,QUEENS,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
2,MANHATTAN,FELONY,DANGEROUS DRUGS
3,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
4,MANHATTAN,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
5,BROOKLYN,FELONY,FELONY ASSAULT
6,MANHATTAN,MISDEMEANOR,DANGEROUS DRUGS
7,BRONX,FELONY,DANGEROUS WEAPONS
8,BRONX,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
9,MANHATTAN,MISDEMEANOR,PETIT LARCENY


In [17]:
#connect to local database
rds_connection_string = "root:Password@127.0.0.1/ny_crime"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [18]:
#check tables
engine.table_names()

['crime_data', 'neighborhoods']

In [19]:
#load borough DataFrame into database
new_group.to_sql(name='neighborhoods', con=engine, if_exists='append', index=False)

In [ ]:
#load crime DataFrame into database
new_crime_pd.to_sql(name='crime_data', con=engine, if_exists='append', index=False)

In [21]:
pd.read_sql_query('select * from neighborhoods', con=engine).head()

,id,BOROUGH,Total_Participants,Total_Pub_Assist,Percent_Pub_Assist
0,26,Bronx,556,187,34.0
1,27,Brooklyn,1067,360,34.0
2,28,Manhattan,127,30,24.0
3,29,Not,2338,824,35.0
4,30,Queens,48,5,10.0


In [22]:
pd.read_sql_query('select * from crime_data', con=engine).head()

,id,BORO_NM,LAW_CAT_CD,OFNS_DESC
0,1,BRONX,FELONY,FORGERY
1,2,QUEENS,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
2,3,MANHATTAN,FELONY,DANGEROUS DRUGS
3,4,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
4,5,MANHATTAN,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES


# MySQL Code

Make Database

`CREATE DATABASE ny_crime;`

`USE ny_crime;`

Make Table and Columns

`CREATE TABLE neighborhoods (
	id INT AUTO_INCREMENT NOT NULL,
    BOROUGH TEXT,
    Total_Participants INT,
    Total_Pub_Assist INT,
    primary key (id)
    );`
    
 Make Table and Columns
 
`CREATE TABLE crime_data (
	id INT auto_increment,
    BORO_NM TEXT,
    LAW_CAT_CD TEXT,
    OFNS_DESC TEXT,
    primary key (id)
    );`

Add Column to Previous Table

`ALTER TABLE neighborhoods
ADD Percent_Pub_Assist DECIMAL AFTER Total_Pub_Assist;`


`SET SQL_SAFE_UPDATES = 1;`
`SELECT * FROM neighborhoods;`

`SELECT * FROM crime_data;`

Merge Tables

`SELECT n.BOROUGH, n.Total_Participants, n.Total_Pub_Assist, n.Percent_Pub_Assist, c.LAW_CAT_CD, c.OFNS_DESC
FROM crime_data AS c
INNER JOIN neighborhoods AS n
ON n.BOROUGH = c.BORO_NM;`
